# Building Functions

Here is my little playground to start writing proper functions for HN

In [38]:
using Graphs
using SimpleWeightedGraphs
using Dice 
using PrettyTables
include("intro.jl")
using DataRefine

using GraphPlot

Currently active project is: HN_V2

Path of active project: /Users/quaidzahid/Documents/Research Professor/Mikhail Erementchouk/HN_V2

Have fun with your new project!

You can help us improve DrWatson by opening
issues on GitHub, submitting feature requests,
or even opening your own Pull Requests!



In [1]:
function get_HN_graph(images ::Vector{Matrix{Int}},
                                scale ::Float64) ::SimpleWeightedGraph
    part_data = vec(images[1]) # Column major vector, columns are coancenated one after the other
    # part data only used to determine the number of entries in matrix
    # We assume that all images are of the same dimension here
    graph_set = SimpleWeightedGraph(length(part_data))
        
    for og in range(1,length(part_data) - 1)
        for term in range(og+1,length(part_data))
            # EDIT HERE, i is origin, j is end. We need to do this multiplication for every image
            # og and term give the values of column major, but that should not affect anything
            w = 0
            for i in images
                w += i[og] * (i[term]) / 1
            end
            add_edge!(graph_set, og, term, w)
        end
    end
    return graph_set
end

get_HN_graph (generic function with 1 method)

In [3]:
function save_trajectories(traj_collection::Vector{Vector{Hybrid}},
                           prefix::String,
                           suffix::String)
    # Save the trajectories in the Spin Reader format
    # traj_collection is a collection of trajectories collected after
    # agitations. We save each agitation in a separate file.
    for (ind, traj) in enumerate(traj_collection)
        out_file_name = prefix * "_$(ind)_" * suffix * ".dat"
        open(out_file_name, "w") do outf
            for (t, state) in enumerate(traj)
                print(outf, "$t")
                for (s, x) in zip(state[1], state[2])
                    print(outf, " $s $x")
                end
                println(outf, "")
            end
        end
        println("File $out_file_name is generated")
    end
    
end

save_trajectories (generic function with 1 method)

In [7]:
#= template

image = [
    1 1 1 1 1;
    1 1 1 1 1;
    1 1 1 1 1;
    1 1 1 1 1;
    1 1 1 1 1;
] 
=#
i1 = [
    -1 1 1 1 -1;
    1 -1 1 -1 1;
    1 1 -1 1 1;
    1 -1 1 -1 1;
    -1 1 1 1 -1;
] 

i2 = [
    1 1 -1 1 1;
    1 -1 -1 -1 1;
    -1 -1 -1 -1 -1;
    1 -1 -1 -1 1;
    1 1 -1 1 1;
] 

i3 = [
    1 -1 1 -1 1;
    -1 -1 -1 -1 -1;
    1 -1 1 -1 1;
    -1 -1 -1 -1 -1;
    1 -1 1 -1 1;
] 

# Params 
# Take image matrix and turn it into a vector with the rows being coancenated at their endpoints
# ex: [row1,row2,row3]

sim_time = 4
steps = 1500
parameters = Dict{String, Any}(
    "images" => [i1,i2],
    "sim_time" => sim_time,
    "steps" => steps,
    "iterations" => 1, #Num of times attempted
    #"num_agitations" => 1, no agitations needed
    "scaling" => 1.0,
    "delta" => -1* sim_time/steps,
    # This is the region that we will provide and the machine must keep this region constant
    # Will be a vector of tuples which are (row, col, state)
    "initial_stimuli" => [(2,2,-1)]
)

Dict{String, Any} with 7 entries:
  "images"          => [[-1 1 … 1 -1; 1 -1 … -1 1; … ; 1 -1 … -1 1; -1 1 … 1 -1…
  "scaling"         => 1.0
  "iterations"      => 1
  "steps"           => 1500
  "initial_stimuli" => [(2, 2, -1)]
  "sim_time"        => 4
  "delta"           => -0.00266667

In [33]:
function HN_Solver(parameters::Dict{String, Any}, debug::Bool = false)
    # Just gets raw result, no visualizer

    scaling = parameters["scaling"] #Scaling coeff
    S = parameters["images"] # Images
    time_total= parameters["sim_time"] # how long the sim is
    num_steps = parameters["steps"] # number of steps in the sim
    delta_t = parameters["delta"] # dt

    # Make Graph
    graph_set = get_HN_graph(S, scaling)
    
    # Making model
    model = Dice.Model(graph_set, Dice.model_2_hybrid_coupling, delta_t)

    # Making randomized initial state
    num_vertices = Graphs.nv(model.graph)
    
    converged = 0
    diverged = 0

    for _ in 1:parameters["iterations"]
        pinned::Vector{Tuple{Int64, Int8}} = []
        state::Dice.Hybrid = Dice.get_random_hybrid(num_vertices, 2.0)
        #println(reshape(state[1],size(parameters["images"][1],1),size(parameters["images"][1],1)))
        for stim in parameters["initial_stimuli"]
            # Every n entries is a column, thus taking # of columns and subtracting 1 brings you to where the column begins
            # Adding 1 will bring you to the first entry in the column and so on for +k
            pos = (stim[2]-1) * size(parameters["images"][1],1) + stim[1]
            state[1][pos] = stim[3]
            push!(pinned, (pos, stim[3]))
        end
        #println(state[1]
        state = Dice.propagate_pinned(model.graph, num_steps, delta_t, model.coupling, state, pinned)
        # Comment out the other part of the or conditional to ignore negative images
        if (reshape(state[1],size(parameters["images"][1],1),size(parameters["images"][1],1)) in parameters["images"] ||
        -1 .*reshape(state[1],size(parameters["images"][1],1),size(parameters["images"][1],1)) in parameters["images"])
            converged += 1
            global sol = state[1]
            break
        end
    end
    
    if debug
        pretty_table(reshape(sol,size(parameters["images"][1],1),size(parameters["images"][1],1)))
        print("Number converged: $(converged) \n Ratio Converged: $(converged/parameters["iterations"])")
    end
    
end

    

HN_Solver (generic function with 2 methods)

In [34]:
HN_Solver(parameters, true)

┌────────┬────────┬────────┬────────┬────────┐
│ Col. 1 │ Col. 2 │ Col. 3 │ Col. 4 │ Col. 5 │
├────────┼────────┼────────┼────────┼────────┤
│      1 │      1 │     -1 │      1 │      1 │
│      1 │     -1 │     -1 │     -1 │      1 │
│     -1 │     -1 │     -1 │     -1 │     -1 │
│      1 │     -1 │     -1 │     -1 │      1 │
│      1 │      1 │     -1 │      1 │      1 │
└────────┴────────┴────────┴────────┴────────┘
Number converged: 1 
 Ratio Converged: 1.0

In [42]:
function HN_Solver_Traj(parameters::Dict{String, Any}, debug::Bool = false)
    # Just gets raw result, no visualizer

    scaling = parameters["scaling"] #Scaling coeff
    S = parameters["images"] # Images
    time_total= parameters["sim_time"] # how long the sim is
    num_steps = parameters["steps"] # number of steps in the sim
    delta_t = parameters["delta"] # dt
    traj_collection::Vector{Vector{Dice.Hybrid}} = []

    # Make Graph
    graph_set = get_HN_graph(S, scaling)
    
    # Making model
    model = Dice.Model(graph_set, Dice.model_2_hybrid_coupling, delta_t)

    # Making randomized initial state
    num_vertices = Graphs.nv(model.graph)
    
    converged = 0
    diverged = 0


    pinned::Vector{Tuple{Int64, Int8}} = []
    state::Dice.Hybrid = Dice.get_random_hybrid(num_vertices, 2.0)
    #println(reshape(state[1],size(parameters["images"][1],1),size(parameters["images"][1],1)))
    for stim in parameters["initial_stimuli"]
        # Every n entries is a column, thus taking # of columns and subtracting 1 brings you to where the column begins
        # Adding 1 will bring you to the first entry in the column and so on for +k
        pos = (stim[2]-1) * size(parameters["images"][1],1) + stim[1]
        state[1][pos] = stim[3]
        push!(pinned, (pos, stim[3]))
    end
    #println(state[1]
    traj = Dice.trajectories_pinned(model.graph, num_steps, delta_t, model.coupling, state, pinned)
    push!(traj_collection, traj)
    # Comment out the other part of the or conditional to ignore negative images
 #=   if (reshape(state[1],size(parameters["images"][1],1),size(parameters["images"][1],1)) in parameters["images"] ||
        -1 .*reshape(state[1],size(parameters["images"][1],1),size(parameters["images"][1],1)) in parameters["images"])
        converged += 1
        global sol = state[1]
        break
    end =#

    
    if debug
        pretty_table(reshape(traj[end][1],size(parameters["images"][1],1),size(parameters["images"][1],1)))
    end
    return((state,traj_collection))
end    

HN_Solver_Traj (generic function with 2 methods)

In [43]:
res = HN_Solver_Traj(parameters, true);
save_trajectories(res[2], "test", "yes")

┌────────┬────────┬────────┬────────┬────────┐
│ Col. 1 │ Col. 2 │ Col. 3 │ Col. 4 │ Col. 5 │
├────────┼────────┼────────┼────────┼────────┤
│     -1 │      1 │      1 │      1 │     -1 │
│      1 │     -1 │      1 │     -1 │      1 │
│      1 │      1 │     -1 │      1 │      1 │
│      1 │     -1 │      1 │     -1 │      1 │
│     -1 │      1 │      1 │      1 │     -1 │
└────────┴────────┴────────┴────────┴────────┘
File test_1_yes.dat is generated
